# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

В ходе исследования необходимо ответить на следующие вопросы:

1) Есть ли зависимость между наличием детей и возвратом кредита в срок?

2) Есть ли зависимость между семейным положением и возвратом кредита в срок?

3) Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

4) Как разные цели кредита влияют на его возврат в срок?

**Описание данных:**

1) children — количество детей в семье

2) days_employed — трудовой стаж в днях

3) dob_days — возраст клиента в годах

4) education — образование клиента

5) education_id — идентификатор образования

6) family_status — семейное положение

7) family_status_id — идентификатор семейного положения

8) gender — пол клиента

9) income_type — тип занятости

10) debt — имел ли задолженность по возврату кредитов

11) total_income — доход в месяц

12) purpose — цель получения кредита



## Шаг 1. Откройте файл с данными и изучите общую информацию. <a id='section1'></a>

In [285]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [286]:
length_original = len(data)
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

В файле с данными data.csv представлена таблица с 21525 строками и 12 столбцами. Из всех столбцов 2 вещественного типа данных float64 (days_employed, total_income), 5 целочисленного типа int64 (children, dob_years, education_id, family_status_id, debt), 5 строкового типа object (education, family_status, gender, income_type, purpose).
В дынных выделяется 2 столбца имеющие пропущенные значения: days_employed, total_income. Кроме того в столбце days_employed присутствуют отрицательные и аномально большие значения.

## Шаг 2. Предобработка данных <a id='section2'></a>

### Обработка пропусков <a id='section3'></a>

Отрицательные значения в столбце 'days_employed' могли появиться вследствии ошибки в формуле, а именно из стажа вычиталась дата получения данных от клиента банка, а следовало сделать наоборот. Изменим значения на положительные.

In [287]:
data['days_employed'] = abs(data['days_employed']) #заменяем значения столбца 'days_employed' на равные по модулю
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Далее проверим есть не превышает ли трудовой стаж возраст клиентов банка. Если будут найдены аномальные значения, тогда мы имеем дело с некоректно внесенными данными, и придется принять допущение, что стаж в таких строках указан не в днях, а в часах.

In [288]:
print(data.loc[data['days_employed']/365 > data['dob_years']].count())

children            3519
days_employed       3519
dob_years           3519
education           3519
education_id        3519
family_status       3519
family_status_id    3519
gender              3519
income_type         3519
debt                3519
total_income        3519
purpose             3519
dtype: int64


В 3519 столбцах стаж превышает возраст клиента банка. Необходимо перезаписать эти значения, при этом переводя их из часов в дни.

In [289]:
print('Количество клиентов с нулевым возрастом: ', data.loc[data['dob_years'] == 0, 'dob_years'].count())

Количество клиентов с нулевым возрастом:  101


In [290]:
data.loc[data['dob_years'] == 0, 'dob_years'] = np.NaN
data = data.dropna(subset=['dob_years']).reset_index(drop = True)
print('Количество клиентов с нулевым возрастом: ', data.loc[data['dob_years'] == 0, 'dob_years'].count())
print("Количество клиентов с NaN'овым возрастом: ", data.loc[data['dob_years'].isna(), 'dob_years'].count())

Количество клиентов с нулевым возрастом:  0
Количество клиентов с NaN'овым возрастом:  0


In [291]:
def hours_in_days(row):
    years = row['days_employed'] / 365
    if years > row['dob_years']:
        row['days_employed'] = row['days_employed'] / 24
    return row['days_employed']

data['days_employed'] = data.apply(hours_in_days, axis=1)
print(data.loc[data['days_employed']/365 > data['dob_years']].count())

children            54
days_employed       54
dob_years           54
education           54
education_id        54
family_status       54
family_status_id    54
gender              54
income_type         54
debt                54
total_income        54
purpose             54
dtype: int64


In [292]:
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27.0,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43.0,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50.0,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35.0,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41.0,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Подсчитаем колличество клиентов представленных в каждой категорий типов занятости.

In [293]:
data_income_type_count = data['income_type'].value_counts()
print(data_income_type_count)

сотрудник          11064
компаньон           5065
пенсионер           3836
госслужащий         1453
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64


Категории 'безработный', 'предприниматель', 'в декрете' и 'студент' непоказательные, так как по ним собранно мало данных. Поэтому удалим их.

In [294]:
import numpy as np

def income_type_fill_by_na(income_type): # Функция присваивающая непоказательным категориям значение NaN
    if (income_type == 'безработный') or (income_type == 'предприниматель') or (income_type == 'в декрете') or (income_type == 'студент'):
        income_type = np.NaN
    return income_type

data['income_type'] = data['income_type'].apply(income_type_fill_by_na)
data = data.dropna(subset=['income_type']).reset_index(drop = True) # Удаление строк содержащих NaN в 'income_type'

print('Пустые ячейки в столбце income_type:', data[data['income_type'] == 'NaN']['income_type'].count())

Пустые ячейки в столбце income_type: 0


Найдем сколько клиентов имеют слишком большой стаж.

In [295]:
data_wrong_days = data.loc[data['days_employed']/365 > data['dob_years']]['income_type'].value_counts()
data_wrong_days

пенсионер    52
Name: income_type, dtype: int64

Подсчитаем конверсию и на основании этих данных примем решение о удалении или переработке имеющихся значений.

In [296]:
conv_wrong_income_type = data_wrong_days / data_income_type_count
conv_wrong_income_type

безработный             NaN
в декрете               NaN
госслужащий             NaN
компаньон               NaN
пенсионер          0.013556
предприниматель         NaN
сотрудник               NaN
студент                 NaN
Name: income_type, dtype: float64

Неправильно внесенные значения стажа из выборки по категориям занятости либо ничтожно малы и не превышают 2%. Поэтому удалим эти строки из наших данных.

In [297]:
def income_type_fill_by_na_wrong_age(row): # Функция присваивающая значение NaN клиентам со слишком большим стажем
    if row['days_employed']/365 > row['dob_years']:
        row['income_type'] = np.NaN
    return row

data = data.apply(income_type_fill_by_na_wrong_age, axis=1)
data = data.dropna(subset=['income_type']).reset_index(drop = True) # Удаление строк содержащих NaN в 'income_type'

print('Количество строк со слишком большим стажем:', data.loc[data['days_employed']/365 > data['dob_years']]['income_type'].count())

Количество строк со слишком большим стажем: 0


Подсчитаем медианную зарплату и трудовой стаж и сгрупируем значения в зависимости от занятости.

In [298]:
data_grouped_days_employed_median = data.groupby('income_type')['days_employed'].median()
data_grouped_income_median = data.groupby('income_type')['total_income'].median()

print('Медианный трудовой стаж:', data_grouped_days_employed_median)
print('Медианная зарплата:', data_grouped_income_median)

Медианный трудовой стаж: income_type
госслужащий     2673.404956
компаньон       1548.637544
пенсионер      15207.350401
сотрудник       1576.067689
Name: days_employed, dtype: float64
Медианная зарплата: income_type
госслужащий    150521.917580
компаньон      172280.581712
пенсионер      118682.554337
сотрудник      142594.396847
Name: total_income, dtype: float64


Подсчитаем количество пропусков в столбцах 'days_employed' и 'total_income'

In [299]:
print("Количество пропусков в стобце 'days_employed':", data['days_employed'].isnull().sum()) 
print("Количество пропусков в стобце 'total_income':", data['total_income'].isnull().sum())

Количество пропусков в стобце 'days_employed': 2163
Количество пропусков в стобце 'total_income': 2163


Так как пропущенных значений слишком много, удалять строки, в которых они представлены, нельзя. Поэтому изменим их на медиану по каждой категории типа занятости, которые были найдены выше.

In [300]:
def fill_null(row):
    income_type_current = row['income_type']
    row['days_employed'] = data_grouped_days_employed_median[income_type_current]
    row['total_income'] = data_grouped_income_median[income_type_current]
    return row

data.loc[data['days_employed'].isnull()] = data.loc[data['days_employed'].isnull()].apply(fill_null, axis=1)
print("Количество пропусков в стобце 'days_employed':", data['days_employed'].isnull().sum()) 
print("Количество пропусков в стобце 'total_income':", data['total_income'].isnull().sum())

Количество пропусков в стобце 'days_employed': 0
Количество пропусков в стобце 'total_income': 0


In [301]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Проверим столбец 'children' на предмет наличия аномальных значений.

In [302]:
data['children'].value_counts()

 0     14039
 1      4792
 2      2037
 3       327
 20       75
-1        46
 4        41
 5         9
Name: children, dtype: int64

Отрицательного количества детей у человека быть не может, поэтому избавимся от строк, которые содержат такие значения.

In [303]:
def children_fill_by_na(value): # Функция присваивающая значение NaN отрицательным значениям
    if value == -1:
        value = np.NaN
    return value

data['children'] = data['children'].apply(children_fill_by_na)
data = data.dropna(subset=['children']).reset_index(drop = True) # Удаление строк содержащих NaN в 'income_type'
data['children'].value_counts()

0.0     14039
1.0      4792
2.0      2037
3.0       327
20.0       75
4.0        41
5.0         9
Name: children, dtype: int64

Проверим столбец 'gender' на предмет наличия аномальных значений.

In [304]:
data['gender'].value_counts()

F      14098
M       7221
XNA        1
Name: gender, dtype: int64

Удалим аномальное значение пола XNA.

In [305]:
data.loc[data['gender'] == 'XNA', 'gender'] = np.NaN
data = data.dropna(subset=['gender']).reset_index(drop = True)
data['gender'].value_counts()

F    14098
M     7221
Name: gender, dtype: int64

In [306]:
length_original
length_after_drop = len(data)
print('После обработки пропусков потеряно {:.2%} строк'.format(1 - length_after_drop / length_original))

После обработки пропусков потеряно 0.96% строк


In [307]:
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1.0,8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1.0,4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0.0,5623.422610,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3.0,4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0.0,14177.753002,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0.0,926.185831,27.0,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0.0,2879.202052,43.0,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0.0,152.779569,50.0,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2.0,6929.865299,35.0,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0.0,2188.756445,41.0,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

В ходе исследования было обнаружено, что многие значения столбца 'days_employed' отрицательные, возможно, вследствии ошибки, совершенной на этапе внесения данных, а именно было произведено вычитание из трудового стажа даты внесения данных, а следовало сделать наоборот. Мы произвели изменение таких вначений на равные по модулю.
Кроме того многие клиенты имели аномально большой трудовой стаж даже больше их возраста. Такие данные могли получиться из-за ошибки в размерности и были внесены значения, указанные не в днях, а в часах. Была произведена конвертация таких значений из часов в дни, а оставшиеся аномально большие значения уже не вносили существенного вклада в общую картину и были удалены.
В данных были обнаружены пропуски в столбцах 'days_employed' и 'total_income', которые были заполнены медианными значениями по соответствующим типам занятости. Также были удалены строки относящиеся к непоказательным типам занятости, а именно 'безработный', 'предприниматель', 'в декрете' и 'студент', так как данных в них собрано мало и их дальнейшее использование может испортить полученные выводы.

### Замена типа данных <a id='section4'></a>

In [308]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21319 entries, 0 to 21318
Data columns (total 12 columns):
children            21319 non-null float64
days_employed       21319 non-null float64
dob_years           21319 non-null float64
education           21319 non-null object
education_id        21319 non-null int64
family_status       21319 non-null object
family_status_id    21319 non-null int64
gender              21319 non-null object
income_type         21319 non-null object
debt                21319 non-null int64
total_income        21319 non-null float64
purpose             21319 non-null object
dtypes: float64(4), int64(3), object(5)
memory usage: 2.0+ MB


В таблице присутствуют 2 столбца вещественного типа float64. Изменим тип данных на целочисленный int64.

In [309]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21319 entries, 0 to 21318
Data columns (total 12 columns):
children            21319 non-null float64
days_employed       21319 non-null int64
dob_years           21319 non-null float64
education           21319 non-null object
education_id        21319 non-null int64
family_status       21319 non-null object
family_status_id    21319 non-null int64
gender              21319 non-null object
income_type         21319 non-null object
debt                21319 non-null int64
total_income        21319 non-null int64
purpose             21319 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Используя метод astype() тип данных в столбцах 'days_employed' и 'total_income' был изменен с float64 на int64. Это делалось для удобства восприятия информации в таблице, потому что дроби с мноеством знаков после запятой рассеивают внимание и мешают анализировать данные.

### Обработка дубликатов <a id='section5'></a>

In [310]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1.0,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1.0,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0.0,5623,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3.0,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0.0,14177,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0.0,926,27.0,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0.0,2879,43.0,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0.0,152,50.0,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2.0,6929,35.0,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0.0,2188,41.0,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


В столбце 'education' присутствуют значения одинаковые по смыслу, но имеющие разный регистр, из-за чего воспринимаются программой как различные. Исправим это, сделав все буквы в столбце строчными.

In [311]:
data['education'] = data['education'].str.lower()
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1.0,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1.0,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0.0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3.0,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0.0,14177,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0.0,926,27.0,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0.0,2879,43.0,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0.0,152,50.0,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2.0,6929,35.0,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0.0,2188,41.0,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


Найжем количество дубликатов в таблице.

In [312]:
print('Количество дубликатов:', data.duplicated().sum())

Количество дубликатов: 71


Удалим имеющиеся дубликаты и восстановим нумерацию индексов строк.

In [313]:
data.drop_duplicates(inplace=True)
data = data.reset_index(drop=True)
print('Количество дубликатов:', data.duplicated().sum())

Количество дубликатов: 0


### Вывод

При первом выводе таблицы на экран было видно, что одинаковые по смыслу значения столбца 'education' могут иметь разный регистр и восприниматься программой как разные. Поэтому перед поиском дубликотов был применен метод str.lower(), который сделал все буквы в столбце 'education' строчными. После этого был выполнен поиск дубликотов и последующее их удаление.

### Лемматизация <a id='section6'></a>

Для того, чтобы в дальнейшем оценить влияние цели кредита на его возврат в срок, нужно провести лемматизацию столбца целей 'purpose'. Для этого напишем функцию, которая принимает строковое значение из ячейки столбца 'purpose', производит его разложение на леммы и записывает их в массив, а далее склеивает этот массив в стоку, удаляя ненужный символ переноса строки '\n'.

In [314]:
from pymystem3 import Mystem
m = Mystem()

def lemmas_of_purpose(purpose):
    return ' '.join(m.lemmatize(purpose)).rstrip('\n')

data['purpose'] = data['purpose'].apply(lemmas_of_purpose)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1.0,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилье
1,1.0,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиль
2,0.0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилье
3,3.0,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительный образование
4,0.0,14177,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьба


### Вывод

Была произведена лемматизация целей получения кредита, чтобы в дальнейшем можно было произвести категоризацию по целям и понять, как цель кредитования влияет на возврат денежных средств в срок.

### Категоризация данных <a id='section7'></a>

Проведем категоризацию целей, для этого проанализируем все цели указанные в столбце 'purpose'.

In [315]:
print(data['purpose'].value_counts())

автомобиль                                     960
свадьба                                        780
на   проведение   свадьба                      759
сыграть   свадьба                              755
операция   с   недвижимость                    669
покупка   коммерческий   недвижимость          658
покупка   жилье   для   сдача                  645
операция   с   жилье                           644
операция   с   коммерческий   недвижимость     643
покупка   жилье   для   семья                  637
покупка   жилье                                637
жилье                                          636
строительство   собственный   недвижимость     631
недвижимость                                   627
операция   со   свой   недвижимость            623
строительство   недвижимость                   618
покупка   свой   жилье                         617
строительство   жилой   недвижимость           617
покупка   недвижимость                         610
ремонт   жилье                 

Из вышеуказанный целей можно выделить 4 основные категории: автомобиль, свадьба, недвижимость и образование. Напишем функцию, которая изменит леммы в столбце 'purpose' на соответствующие им назнания категорий.

In [316]:
def purpose_normalize(purpose):
    if 'автомобиль' in purpose:
        return 'автомобиль'
    if 'свадьба' in purpose:
        return 'свадьба'
    if ('недвижимость' in purpose) or ('жилье' in purpose):
        return 'недвижимость'
    if 'образование' in purpose:
        return 'образование'

data['purpose'] = data['purpose'].apply(purpose_normalize)
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1.0,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость
1,1.0,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,0.0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость
3,3.0,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,0.0,14177,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба
5,0.0,926,27.0,высшее,0,гражданский брак,1,M,компаньон,0,255763,недвижимость
6,0.0,2879,43.0,высшее,0,женат / замужем,0,F,компаньон,0,240525,недвижимость
7,0.0,152,50.0,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2.0,6929,35.0,высшее,0,гражданский брак,1,F,сотрудник,0,95856,свадьба
9,0.0,2188,41.0,среднее,1,женат / замужем,0,M,сотрудник,0,144425,недвижимость


Оценим корректность такой категоризации, подсчитав количество клиентов представленных в каждой из категорий.

In [317]:
print(data['purpose'].value_counts())

недвижимость    10715
автомобиль       4267
образование      3972
свадьба          2294
Name: purpose, dtype: int64


В каждой категории представленно достаточное количество данных для подсчета в дальнейшем корректных результатов. Теперь проиндексируем данные категории через соответствующую функцию и создадим словарь целей, который будет включать в себя цели и их индексы.

In [318]:
def purpose_to_id(value):
    if value == 'автомобиль':
        return 0
    if value == 'свадьба':
        return 1
    if value == 'недвижимость':
        return 2
    if value == 'образование':
        return 3

purpose_dict = data[['purpose']]
purpose_dict = purpose_dict.drop_duplicates() # удаление дубликатов
purpose_dict['purpose_id'] = purpose_dict['purpose'].apply(purpose_to_id)
purpose_dict = purpose_dict.sort_values('purpose_id', ascending=True).reset_index(drop = True)# сортировка по возрастанию id с востановлением индексов
purpose_dict

,purpose,purpose_id
0,автомобиль,0
1,свадьба,1
2,недвижимость,2
3,образование,3


Объединим изначальную таблицу с данными и словарь целей.

In [319]:
data = data.merge(purpose_dict, on='purpose', how='left')
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_id
0,1.0,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,2
1,1.0,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,0
2,0.0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость,2
3,3.0,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,3
4,0.0,14177,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,1


Проведем категоризацию типа занятости, для этого узнаем все представленные категории занятости.

In [320]:
data['income_type'].value_counts()

сотрудник      11003
компаньон       5048
пенсионер       3750
госслужащий     1447
Name: income_type, dtype: int64

Проиндекцируем все вышеуказанные категории и создадим словарь с категориями по типу занятости.

In [321]:
def income_type_to_id(value):
    if value == 'сотрудник':
        return 0
    if value == 'компаньон':
        return 1
    if value == 'пенсионер':
        return 2
    if value == 'госслужащий':
        return 3

income_type_dict = data[['income_type']]
income_type_dict = income_type_dict.drop_duplicates()
income_type_dict['income_type_id'] = income_type_dict['income_type'].apply(income_type_to_id)
income_type_dict.sort_values('income_type_id', ascending=True).reset_index(drop = True)

,income_type,income_type_id
0,сотрудник,0
1,компаньон,1
2,пенсионер,2
3,госслужащий,3


Объединим изначальную таблицу с данными и словарь типов занятости.

In [322]:
data = data.merge(income_type_dict, on='income_type', how='left')
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_id,income_type_id
0,1.0,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,2,0
1,1.0,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,0,0
2,0.0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость,2,0
3,3.0,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,3,0
4,0.0,14177,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,1,2


Проведем категоризацию трудового стажа, для этого примем 4 категории: опыт работы меньше года, от 1 до 5 лет, от 5 до 10 лет и больше 10 лет. Напишем функцию, с помощью которой создадим в таблице столбец с названием категорий.

In [323]:
def days_employed_to_category(value):
    year = 365
    if value < year:
        return 'меньше года'
    if year <= value < year*5:
        return '1-5 лет'
    if year*5 <= value < year*10:
        return '5-10 лет'
    if value >= year*10:
        return 'больше 10 лет'

data['experience'] = data['days_employed'].apply(days_employed_to_category)
data['experience'].value_counts()

1-5 лет          8315
больше 10 лет    6881
5-10 лет         4235
меньше года      1817
Name: experience, dtype: int64

Проиндекцируем все вышеуказанные категории и запишем все полученные значения в столбец индексов опыта работы 'experience_id'.

In [324]:
def experience_to_id(value):
    if value == 'меньше года':
        return 0
    if value == '1-5 лет':
        return 1
    if value == '5-10 лет':
        return 2
    if value == 'больше 10 лет':
        return 3
   
data['experience_id'] = data['experience'].apply(experience_to_id)
data['experience_id'].head(10)

0    3
1    3
2    3
3    3
4    3
5    1
6    2
7    0
8    3
9    2
Name: experience_id, dtype: int64

Создадим словарь категорий опыта работы, в который запишем категории опыты работы и соответствующие им индексы.

In [325]:
experience_dict = data[['experience', 'experience_id']]
experience_dict = experience_dict.drop_duplicates().sort_values('experience_id', ascending=True).reset_index(drop = True)
experience_dict

,experience,experience_id
0,меньше года,0
1,1-5 лет,1
2,5-10 лет,2
3,больше 10 лет,3


Создадим словарь категорий уровня образования, в который запишем категории уровня оразования и соответствующие им индексы. Так как эти данные уже представлены в изначальной тоблице, то никаких жополнительных действий не требуется.

In [326]:
education_dict = data[['education', 'education_id']]
education_dict = education_dict.drop_duplicates().sort_values('education_id', ascending=True).reset_index(drop=True)
education_dict

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


То же самое проделаем со словарем для семейного положения.

In [327]:
family_status_dict = data[['family_status', 'family_status_id']]
family_status_dict = family_status_dict.drop_duplicates().sort_values('family_status_id', ascending=True).reset_index(drop=True)
family_status_dict

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


Проведем категоризацию уровня месячного дохода, для этого примем 4 категории: "низкий" - при доходе до 100000, "средний" - при доходе от 100000 до 150000, "выше среднего" - при доходе от 150000 до 200000 и "высокий" - при доходе от 200000. Напишем функцию, с помощью которой создадим в таблице столбец с названием категорий.

In [328]:
def total_income_to_category(value):
    if value < 100000:
        return 'низкий'
    if 100000 <= value < 150000:
        return 'средний'
    if 150000 <= value < 200000:
        return 'выше среднего'
    if value >= 200000:
        return 'высокий'

data['income_level'] = data['total_income'].apply(total_income_to_category)
data['income_level'].value_counts()

средний          7097
высокий          5019
выше среднего    4722
низкий           4410
Name: income_level, dtype: int64

Проиндекцируем все вышеуказанные категории и запишем все полученные значения в столбец индексов уровня месячного дохода 'income_level_id'.

In [329]:
def income_level_to_id(value):
    if value == 'низкий':
        return 0
    if value == 'средний':
        return 1
    if value == 'выше среднего':
        return 2
    if value == 'высокий':
        return 3
    
data['income_level_id'] = data['income_level'].apply(income_level_to_id)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_id,income_type_id,experience,experience_id,income_level,income_level_id
0,1.0,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,2,0,больше 10 лет,3,высокий,3
1,1.0,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,0,0,больше 10 лет,3,средний,1
2,0.0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость,2,0,больше 10 лет,3,средний,1
3,3.0,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,3,0,больше 10 лет,3,высокий,3
4,0.0,14177,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,1,2,больше 10 лет,3,выше среднего,2


Создадим словарь категорий уровня месячного дохода, в который запишем категории уровня дохода и соответствующие им индексы.

In [330]:
income_level_dict = data[['income_level', 'income_level_id']]
income_level_dict = income_level_dict.drop_duplicates().sort_values('income_level_id', ascending=True).reset_index(drop=True)
income_level_dict

,income_level,income_level_id
0,низкий,0
1,средний,1
2,выше среднего,2
3,высокий,3


Проведем категоризацию количества детей у клиентов, для этого примем 3 категории: "нет детей", "1 ребенок", "От 2 детей и более". Напишем функцию, с помощью которой создадим в таблице столбец с названием категорий, и оценим корректность выбора именно таких категорий, подсчитав количество данных представленных в каждой из них.

In [331]:
def chindren_to_category(value):
    if value == 0:
        return 'нет детей'
    if value == 1:
        return '1 ребенок'
    if value >= 2:
        return 'От 2 детей и более'

data['children_number'] = data['children'].apply(chindren_to_category)
#data.head()
print(data['children_number'].value_counts())

нет детей             13980
1 ребенок              4782
От 2 детей и более     2486
Name: children_number, dtype: int64


Так как категории корректны, проиндексируем данные категории и создадим столец 'children_id' в таблице.

In [332]:
def children_to_id(value):
    if value == 'нет детей':
        return 0
    if value == '1 ребенок':
        return 1
    if value == 'От 2 детей и более':
        return 2
    
data['children_id'] = data['children_number'].apply(children_to_id)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_id,income_type_id,experience,experience_id,income_level,income_level_id,children_number,children_id
0,1.0,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,2,0,больше 10 лет,3,высокий,3,1 ребенок,1
1,1.0,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,0,0,больше 10 лет,3,средний,1,1 ребенок,1
2,0.0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость,2,0,больше 10 лет,3,средний,1,нет детей,0
3,3.0,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,3,0,больше 10 лет,3,высокий,3,От 2 детей и более,2
4,0.0,14177,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,1,2,больше 10 лет,3,выше среднего,2,нет детей,0


Создадим словарь категорий количества детей у клиентов, в который запишем категории числа детей и соответствующие им индексы.

In [333]:
chindren_dict = data[['children_number', 'children_id']]
chindren_dict = chindren_dict.drop_duplicates().sort_values('children_id', ascending=True).reset_index(drop=True)
chindren_dict

,children_number,children_id
0,нет детей,0
1,1 ребенок,1
2,От 2 детей и более,2


Создадим "словарь словарей", чтобы обращаться к нему при дальнейшем поиске взаимосвязей долга и различных категорий данных.

In [334]:
dictionaries = {'children' : chindren_dict,
                'experience' : experience_dict,
                'education' : education_dict,
                'family_status' : family_status_dict,
                'income_type' : income_type_dict,
                'income_level' : income_level_dict,
                'purpose' : purpose_dict}

Создадим таблицу в которой представленны только индексы различных категорий. К ней мы будем обращаться, когда будем искать ответы на вопросы.

In [335]:
data_final = data[['gender','children_id', 'experience_id', 'education_id', 'family_status_id', 'income_type_id', 'income_level_id', 'purpose_id', 'debt']]
data_final.head(10)

,gender,children_id,experience_id,education_id,family_status_id,income_type_id,income_level_id,purpose_id,debt
0,F,1,3,0,0,0,3,2,0
1,F,1,3,1,0,0,1,0,0
2,M,0,3,1,0,0,1,2,0
3,M,2,3,1,0,0,3,3,0
4,F,0,3,1,1,2,2,1,0
5,M,0,1,0,1,1,3,2,0
6,F,0,2,0,0,1,3,2,0
7,M,0,0,1,0,0,1,3,0
8,F,2,3,0,1,0,0,1,0
9,M,0,2,1,0,0,1,2,0


### Вывод

В данных было выделено 7 словарей: 'children' - классификация по количчеству детей, 'experience' - по трудовому стажу, 'education' - по уровню образования, 'family_status' - по семейному положению, 'income_type' - по типу занятости, 'income_level' - по уровню месячного дохода, 'purpose' - по цели получения кредита. Все они были собраны в один словарь "dictionaries" для удобства использования. Хотя все найденные словари не нужны для ответа на вопросы, которые представлены ниже, их наличие оставляет возможности для более точного оценивания рейтинга надежности заемщика.

## Шаг 3. Ответьте на вопросы <a id='section8'></a>

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Выведим словарь категорий по количеству детей.

In [336]:
dictionaries['children']

,children_number,children_id
0,нет детей,0
1,1 ребенок,1
2,От 2 детей и более,2


Создадим сводную таблицу, где столбцами будет пол, строками индексы категорий количества детей у клиента, а значениями внутри таблицы будет количество должников. Далее для удобства восприятия информации объединим словарь с данной таблицей по индексу количества детей. Подсчитаем полное количество должников в каждой категории, полное количество клиентов в каждой категории  и их конверсию.

In [337]:
debt_from_children = data_final.pivot_table(index='children_id', columns='gender', values='debt', aggfunc='sum')
debt_from_children = dictionaries['children'].merge(debt_from_children, on='children_id', how='left')
debt_from_children['total_debt'] = debt_from_children['F'] + debt_from_children['M']
debt_from_children['total_clients'] = data_final['children_id'].value_counts()
debt_from_children['ratio'] = (debt_from_children['total_debt'] / debt_from_children['total_clients'])\
                                                                                    .apply(lambda x: "{0:.2%}".format(x))
debt_from_children

,children_number,children_id,F,M,total_debt,total_clients,ratio
0,нет детей,0,587,467,1054,13980,7.54%
1,1 ребенок,1,241,197,438,4782,9.16%
2,От 2 детей и более,2,154,77,231,2486,9.29%


Отсортируем полученную таблицу в порядке возрастания конверсии и проанализируем результат. Чем выше строка расположена в таблице, тем надежнее категория заемщиков.

In [338]:
debt_from_children.sort_values('ratio', ascending=True)

,children_number,children_id,F,M,total_debt,total_clients,ratio
0,нет детей,0,587,467,1054,13980,7.54%
1,1 ребенок,1,241,197,438,4782,9.16%
2,От 2 детей и более,2,154,77,231,2486,9.29%


### Вывод

Исходя из полученных данных, можно утверждать, что клиенты не имеющие детей самые надежные заемщики, которые вероятнее не будут иметь долгов по кредиту (количество должников составляет всего 7,5% от всей группы). Далее по рейтингу надежности идут клиенты с 1 ребенком (9,16%) и с 2 и более детьми (9,29%), но разница между ними крайне мала и составляет чуть больше 0,1%, поэтому можно не разделять их на различные категории и рассматривать как единое целое.

In [339]:
debt_from_children = data_final.pivot_table(index='children_id', columns='gender', values='debt', aggfunc='sum')
debt_from_children = dictionaries['children'].merge(debt_from_children, on='children_id', how='left')
debt_from_children['total_debt'] = debt_from_children['F'] + debt_from_children['M']
debt_from_children['total_clients'] = data_final['children_id'].value_counts()
debt_from_children['ratio'] = (debt_from_children['total_debt'] / debt_from_children['total_clients']) \
                                                                                    .apply(lambda x: "{0:.2%}".format(x))
debt_from_children

,children_number,children_id,F,M,total_debt,total_clients,ratio
0,нет детей,0,587,467,1054,13980,7.54%
1,1 ребенок,1,241,197,438,4782,9.16%
2,От 2 детей и более,2,154,77,231,2486,9.29%


- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Выведим словарь категорий по семейному положению.

Выведим словарь категорий по количеству детей.

In [340]:
dictionaries['family_status']

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


Создадим сводную таблицу, где столбцами будет пол, строками индексы категорий семейного положения, а значениями внутри таблицы будет количество должников. Далее для удобства восприятия информации объединим словарь с данной таблицей по индексу семейного положения. Подсчитаем полное количество должников в каждой категории, полное количество клиентов в каждой категории и их конверсию.

In [341]:
debt_from_family = data_final.pivot_table(index='family_status_id', columns='gender', values='debt', aggfunc='sum')
debt_from_family = dictionaries['family_status'].merge(debt_from_family, on='family_status_id', how='left')
debt_from_family['total_debt'] = debt_from_family['F'] + debt_from_family['M']
debt_from_family['total_clients'] = data_final['family_status_id'].value_counts()
debt_from_family['ratio'] = (debt_from_family['total_debt'] / debt_from_family['total_clients'])\
                                                                                    .apply(lambda x: "{0:.2%}".format(x))
debt_from_family

,family_status,family_status_id,F,M,total_debt,total_clients,ratio
0,женат / замужем,0,523,398,921,12239,7.53%
1,гражданский брак,1,229,154,383,4103,9.33%
2,вдовец / вдова,2,51,11,62,949,6.53%
3,в разводе,3,61,24,85,1177,7.22%
4,Не женат / не замужем,4,118,154,272,2780,9.78%


Отсортируем полученную таблицу в порядке возрастания конверсии и проанализируем результат. Чем выше строка расположена в таблице, тем надежнее категория заемщиков.

In [342]:
debt_from_family.sort_values('ratio', ascending=True)

,family_status,family_status_id,F,M,total_debt,total_clients,ratio
2,вдовец / вдова,2,51,11,62,949,6.53%
3,в разводе,3,61,24,85,1177,7.22%
0,женат / замужем,0,523,398,921,12239,7.53%
1,гражданский брак,1,229,154,383,4103,9.33%
4,Не женат / не замужем,4,118,154,272,2780,9.78%


### Вывод

Исходя из полученных данных, можно утверждать, что вдовецы и вдовы самые надежные заемщики, которые вероятнее не будут иметь долгов по кредиту (количество должников составляет всего 6,5% от всей группы). Далее по рейтингу надежности идут клиенты в разводе (7,2%). Самая многочисленная группа берущая кредиты, а именно женатые клиенты или находящиеся замужем, стоят на 3 месте и имеют рейтинг (7,5%) не сильно хуже, чем у предыдущей группы. Клиенты, находящиеся в гражданском браке (9,3%) и не женатые (9,8%) имеют наихудшие рейтинги надежности по данной классификации.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Выведим словарь категорий по уровню доходов.

In [343]:
dictionaries['income_level']

,income_level,income_level_id
0,низкий,0
1,средний,1
2,выше среднего,2
3,высокий,3


Создадим сводную таблицу, где столбцами будет пол, строками индексы категорий уровня доходов у клиента, а значениями внутри таблицы будет количество должников. Далее для удобства восприятия информации объединим словарь с данной таблицей по индексу уровня доходов. Подсчитаем полное количество должников в каждой категории, полное количество клиентов в каждой категории  и их конверсию.

In [344]:
debt_from_income_level = data_final.pivot_table(index='income_level_id', columns='gender', values='debt', aggfunc='sum')
debt_from_income_level = dictionaries['income_level'].merge(debt_from_income_level, on='income_level_id', how='left')
debt_from_income_level['total_debt'] = debt_from_income_level['F'] + debt_from_income_level['M']
debt_from_income_level['total_clients'] = data_final['income_level_id'].value_counts()
debt_from_income_level['ratio'] = (debt_from_income_level['total_debt'] / debt_from_income_level['total_clients'])\
                                                                                    .apply(lambda x: "{0:.2%}".format(x))
debt_from_income_level

,income_level,income_level_id,F,M,total_debt,total_clients,ratio
0,низкий,0,251,99,350,4410,7.94%
1,средний,1,367,248,615,7097,8.67%
2,выше среднего,2,200,201,401,4722,8.49%
3,высокий,3,164,193,357,5019,7.11%


Отсортируем полученную таблицу в порядке возрастания конверсии и проанализируем результат. Чем выше строка расположена в таблице, тем надежнее категория заемщиков.

In [345]:
debt_from_income_level.sort_values('ratio', ascending=True)

,income_level,income_level_id,F,M,total_debt,total_clients,ratio
3,высокий,3,164,193,357,5019,7.11%
0,низкий,0,251,99,350,4410,7.94%
2,выше среднего,2,200,201,401,4722,8.49%
1,средний,1,367,248,615,7097,8.67%


### Вывод

Исходя из полученных данных, можно утверждать, что клиенты с высоким уровнем доходов (больше 200000) самые надежные заемщики, которые вероятнее не будут иметь долгом по кредиту (количество должников составляет всего 7,1% от всей группы). Далее контринтуитивно по рейтингу надежности следуют клиенты с уровнем доходов до 100000 (7,9%). На 3 месте находятся клиенты с доходами от 150000 до 200000 (8,5%). И на последнем месте находится самая многочисленная группа клиентов с доходами от 100000 до 150000 (8,7%).

- Как разные цели кредита влияют на его возврат в срок?

Выведим словарь категорий по целям получения кредита.

In [346]:
dictionaries['purpose']

,purpose,purpose_id
0,автомобиль,0
1,свадьба,1
2,недвижимость,2
3,образование,3


Создадим сводную таблицу, где столбцами будет пол, строками индексы категорий по целям получения кредита, а значениями внутри таблицы будет количество должников. Далее для удобства восприятия информации объединим словарь с данной таблицей по индексу целей получения кредита. Подсчитаем полное количество должников в каждой категории, полное количество клиентов в каждой категории  и их конверсию.

In [347]:
debt_from_purpose = data_final.pivot_table(index='purpose_id', columns='gender', values='debt', aggfunc='sum')
debt_from_purpose = dictionaries['purpose'].merge(debt_from_purpose, on='purpose_id', how='left')
debt_from_purpose['total_debt'] = debt_from_purpose['F'] + debt_from_purpose['M']
debt_from_purpose['total_clients'] = data_final['purpose_id'].value_counts()
debt_from_purpose['ratio'] = (debt_from_purpose['total_debt'] / debt_from_purpose['total_clients'])\
                                                                                    .apply(lambda x: "{0:.2%}".format(x))
debt_from_purpose

,purpose,purpose_id,F,M,total_debt,total_clients,ratio
0,автомобиль,0,229,167,396,4267,9.28%
1,свадьба,1,105,77,182,2294,7.93%
2,недвижимость,2,443,335,778,10715,7.26%
3,образование,3,205,162,367,3972,9.24%


Отсортируем полученную таблицу в порядке возрастания конверсии и проанализируем результат. Чем выше строка расположена в таблице, тем надежнее категория заемщиков.

In [349]:
debt_from_purpose.sort_values('ratio', ascending=True)

,purpose,purpose_id,F,M,total_debt,total_clients,ratio
2,недвижимость,2,443,335,778,10715,7.26%
1,свадьба,1,105,77,182,2294,7.93%
3,образование,3,205,162,367,3972,9.24%
0,автомобиль,0,229,167,396,4267,9.28%


### Вывод

Исходя из полученных данных, можно утверждать, что клиенты, получающие кредит на операции с недвижимостью самые надежные заемщики, которые вероятнее не будут иметь долгом по кредиту (количество должников составляет 7,25% от всей группы), также эта группа клиентов самая многочисленная. Далее по рейтингу надежности идут клиенты, получающие кредит на свадьбу (7,9%). На 3 месте клиенты, получающие кредит на образование (9,2%). И на последнем месте находятся клиенты с целью купить автомобиль или провести какие-либо манипуляции с ним (9,3%). Две последние категории схожи конверсии, поэтому при нахождении рейтинга надежности заемщика их можно не разделять.

## Шаг 4. Общий вывод <a id='section9'></a>

В проекте "Исследование надежности заемщиков" были проведены следующие мероприятия: определены и удалены/заполнены пропуски, удалены дубликаты, проведена лемматизация целей получения кредита и проведена категоризация данных. После всех проделанной работы можно наверняка выделить самый надежный заемщиков и тех, останется должен. На этапе ответов на вопросы выяснилось, что самыми надежными клиентами являются бездетные вдовцы/вдовы с высоким уровнем доходов, собирающиеся совершить какую-либо операцию с недвижимостью, а самыми не надежными многодетные не женатые/не замужние клиенты с уровнем доходов от 100000 до 150000 и целью совершить операцию с автомобем.
Портрет среднестастического заемщика выглядит следующим образом: нет детей, женат/замужем, с доходами от 100000 до 150000 и целью совершить операцию с недвижимостью.